<!-- ![FOTO](./img/logo.jpeg)  -->
# ***Deep Vision para tareas de clasificación***
## ***By Noelia Otazo Rojo***

### Objetivos
**Evaluar y comparar** estrategias para la **clasificación de imágenes**, la solución deberá estar basada en el **aprendizaje profundo (Redes Neuronales Convolucionales CNNs)**.

### Contenidos
1.   **Carga** del conjunto de datos
2.   **Inspección** del conjunto de datos
3.   **Acondicionamiento** del conjunto de datos
4.   Desarrollo de la **arquitectura** de red neuronal y **entrenamiento** de la solución
5.   **Monitorización** del proceso de **entrenamiento** para la toma de decisiones
6.   **Evaluación** del modelo predictivo.

### Fuente de datos
Enlace al dataset que voy a utilizar para llevar a cabo el siguiente proyecto:
[Cancer Detection](https://www.kaggle.com/c/histopathologic-cancer-detection/data)



## 0. Preparación del proyecto.
En esta sección añadimeros las sentencias de instalación y las importaciones de las librerías necesarias para el desarrollo del proyecto.

In [ ]:
# Numpy
import numpy as np
from numpy.random import seed

# OpenCV to process images
import cv2

# Directory indexes
from os import listdir

import shutil
import random
from pathlib import Path
from pprint import pprint
from collections import defaultdict

# Matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# Pandas
import pandas as pd

# Tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Conv2D, Activation, Flatten, Dense, Dropout, BatchNormaliz
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Sklearn
from sklearn.utils import class_weight
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit

# Ploteado del esquema gráfico del modelo
from keras.utils.vis_utils import plot_model

## 1. Carga del conjunto de datos.
En esta sección cargaremos el contenido del dataset para su posterior uso.

### 1.1 Carga del conjunto de datos.


In [ ]:
# Data
test_data_path = 'histopathologic-cancer-detection/test'
train_data_path = 'histopathologic-cancer-detection/train'
train_labels_path = pd.read_csv('histopathologic-cancer-detection/train_labels.csv')
sample_submission_path = pd.read_csv('histopathologic-cancer-detection/sample_submission.csv')
class_names = [str(p.name) for p in Path('data/original').glob('*')]

### 1.2 Establecemos los parámetros
Mediante modificaciones podremos ir probando a ver cual se ajusta más.

In [ ]:
# Params 
# Train 
batch_size = 8
monitor = 'val_loss'
learning_rate = 1e-4
epochs = 50
early_stopping_patience = 4
train_backbone = True
version = 0
plateau_factor = 0.5
plateau_patience = 2

# Model 
input_shape = (224, 224, 3)
model_name = 'NOR-HISTOPATHOLOGIC-CANCER-DETECTION'